<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleTwitterDiaster/T2E2_DiasterTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import nltk
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [38]:
df_data = pd.read_csv('/content/drive/MyDrive/DS_data/TwitterDisaster/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/DS_data/TwitterDisaster/test.csv')

In [39]:
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

df_train_labels = df_train.pop('target')
df_valid_labels = df_valid.pop('target')

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

lenght of train is 7537
lenght of valid is 76


In [40]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [41]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped  = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return stripped

## Vectorizlize Text

In [64]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
vectorize_layer.adapt(df_train['text'].to_numpy())

### Build  Model

In [76]:
embedding_dim = 16

model = tf.keras.Sequential([
  vectorize_layer,                           
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

#model.summary()

In [79]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics='accuracy')

epochs = 20
history = model.fit(
    df_train['text'].to_numpy(), 
    df_train_labels,
    epochs=epochs)

Epoch 1/20
236/236 [==============================] - 2s 6ms/step - loss: 0.5251 - accuracy: 0.7000
Epoch 2/20
236/236 [==============================] - 1s 6ms/step - loss: 0.5182 - accuracy: 0.7060
Epoch 3/20
236/236 [==============================] - 1s 6ms/step - loss: 0.5030 - accuracy: 0.7291
Epoch 4/20
236/236 [==============================] - 1s 6ms/step - loss: 0.4881 - accuracy: 0.7437
Epoch 5/20
236/236 [==============================] - 1s 6ms/step - loss: 0.4766 - accuracy: 0.7540
Epoch 6/20
236/236 [==============================] - 2s 6ms/step - loss: 0.4590 - accuracy: 0.7661
Epoch 7/20
236/236 [==============================] - 1s 6ms/step - loss: 0.4472 - accuracy: 0.7761
Epoch 8/20
236/236 [==============================] - 1s 6ms/step - loss: 0.4435 - accuracy: 0.7833
Epoch 9/20
236/236 [==============================] - 1s 6ms/step - loss: 0.4331 - accuracy: 0.7935
Epoch 10/20
236/236 [==============================] - 1s 6ms/step - loss: 0.4247 - accuracy: 0.7950

## Evaluate the model

In [80]:
loss, accuracy = model.evaluate(df_valid['text'], df_valid_labels)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

3/3 [==============================] - 0s 4ms/step - loss: 0.4526 - accuracy: 0.7895
Loss:  0.45255038142204285
Accuracy:  0.7894737124443054


## Prediction

In [92]:
prediction_model = tf.keras.Sequential([model, 
                                        layers.Activation('sigmoid')])

predictions = prediction_model.predict(df_test['text'].to_numpy())



AttributeError: ignored

In [94]:
res = pd.DataFrame(data = {"id" : np.arange(1, 1+len(predictions)) , 
                           "target"   : list(map(lambda x: 0 if x<0.5 else 1, predictions))})

res.to_csv("/content/drive/MyDrive/output/Kaggle_twitterDisaster.csv", index=False)